# Expecting the Unexpected

- programmers have to deal with three types of errors:
    1. Syntax Error
    2. Logical Error
    3. Run-time Error
    
- systems built with software can be fragile due to mostly run-time error
- while the software is highly predictable, the runtime context can provide unexpected inputs and situations
- **exceptions** - run-time error raised when a normal response is impossible

- learn:
    1. how to cause an exception to occur
    2. how to recover when an exception has occured
    3. how to handle different exceptions types in different ways
    4. cleaning up when an exception has occured
    5. creating new types of exception
    6. using the exception syntax for flow control

## Raising exceptions

- an exception is an object raised, that interrupts the sequential execution of statements
- all exceptions derive from *BaseException*
- let's see some exceptions by doing some silly stuff

In [1]:
print "hello world"

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (3923495743.py, line 1)

In [2]:
x = 5/0

ZeroDivisionError: division by zero

In [3]:
lst = [1, 2, 3]
print(lst[3])

IndexError: list index out of range

In [4]:
lst.add

AttributeError: 'list' object has no attribute 'add'

In [6]:
lst+10

TypeError: can only concatenate list (not "int") to list

In [8]:
d = {'a': 1}

In [9]:
d ['b']

KeyError: 'b'

In [10]:
print(some_var)

NameError: name 'some_var' is not defined

## Explictly raising an exception

- at times, you may have to explictly raise exceptions in your code for some unexpected situation
- we can use the exact same mechanism, that Python uses
- the following is a simple class adds item to a list if it's an even numbered integer

In [11]:
from typing import List

class EvenOnly(List[int]):
    def append(self, value: int) -> None:
        if not isinstance(value, int):
            raise TypeError(f'Not an integer: {value}')
        if value % 2 != 0:
            raise ValueError(f'Not an even integer: {value}')
        super().append(value)


In [12]:
e = EvenOnly()

In [13]:
e.append(4)

In [14]:
e.append(5)

ValueError: Not an even integer: 5

In [15]:
e.append('a string')

TypeError: Not an integer: a string

In [18]:
# EvenOnly inherits all the methods from List
e.insert(0, 5)

In [19]:
e

[5, 4]

- additional methods/behaviors need to be overriden to make EvenOnly more robust/complete in what it does
    - e.g., insert(), extend(), __setitem__(), __init__(), etc.

## The effects of an exception

- when an exception is raised, program halts at the line number
- the code after the problem statement will not be executed, unless the exception is handled by an exception clause

In [20]:
from typing import NoReturn

def never_returns() -> NoReturn:
    print('Before exception is raised')
    raise Exception('This is always raised')
    print('This line will never be executed')
    return "This will never be returned"

In [21]:
never_returns()

Before exception is raised


Exception: This is always raised

In [22]:
# called within the function
def call_exceptor() -> None:
    print('call_exceptor starts here...')
    never_returns()
    print('an exceptin was raised')
    print("...so these lines don't run")

In [23]:
call_exceptor()

call_exceptor starts here...
Before exception is raised


Exception: This is always raised

## Handling exceptions

- use try and except blocks
- try statement has several separate clauses/parts
- can also handle multiple excetions within the same except block

```python

    try:
        # statement(s) thay may potentially raise an exception
    except ExceptionName1:
        # catch/handle specific exception ExceptionName1
        # statement(s) to handle the exception
    [except ExceptionName2 as err:
        # statements
    ]
    [except:
        # catch any error not caught by previous except blocks
    ]
    [else:
        # follows all except clause
        # executes if try clause does NOT raise an exception
    ]
    [finally:
        # clean-up actions that must be executed under all circumstances; 
        # exectued on the way out when try block is left via a break, continue, or return statement
    ]

```
- [ ... ] optional
- finally clause can be used for:
    - cleaning up an open database connection
    - closing an open file
    - sending a closing handshake over the network

In [26]:
from typing import Union

def funnier_divison(divisor: int) -> Union[str, float]:
    try:
        if divisor == 13:
            raise ValueError("13 is an unlucky number")
        return 100 / divisor
    except (ZeroDivisionError, TypeError):
        return "Enter a number other than zero."

In [27]:
for val in (0, 'hello', 50.0, 3, 13):
    print(f'Testing {val!r:}', end=' ')
    print(funnier_divison(val))

Testing 0 Enter a number other than zero.
Testing 'hello' Enter a number other than zero.
Testing 50.0 2.0
Testing 3 33.333333333333336
Testing 13 

ValueError: 13 is an unlucky number

In [30]:
def funniest_division(divisor: int) -> Union[str, float]:
    try:
        if divisor == 13:
            raise ValueError('13 is an unlucky number')
        return 100 / divisor
    except ZeroDivisionError as ex:
        return f'{ex}: Enter a number other than zero'
    except TypeError as ex:
        return f'{ex}: Enter a numerical value'
    except ValueError: 
        print('No, No, not 13!')
        raise # re-raise the ValueError
        

In [39]:
# let's test funniest_division with some values
divisors = (10, 5, 0, 'hi', 13)
for divisor in divisors:
    print(f'funniest_division({divisor!r})=', funniest_division(divisor))

funniest_division(10)= 10.0
funniest_division(5)= 20.0
funniest_division(0)= division by zero: Enter a number other than zero
funniest_division('hi')= unsupported operand type(s) for /: 'int' and 'str': Enter a numerical value
No, No, not 13!


ValueError: 13 is an unlucky number

In [32]:
# example of finally and else clauses

some_exceptions = [ValueError, TypeError, IndexError, None]

for ex in some_exceptions:
    try:
        print(f'\nRaising {ex}')
        if ex:
            raise ex("An error")
        else:
            print('no exception raised')
    except ValueError:
        print('Caught a ValueError')
    except TypeError:
        print('Caught a TypeError')
    except Exception as e:
        print(f'Caught some other error: {e.__class__.__name__}')
    else:
        print('this code called if there is no exception')
    finally:
        print('this clean up code is always called')


Raising <class 'ValueError'>
Caught a ValueError
this clean up code is always called

Raising <class 'TypeError'>
Caught a TypeError
this clean up code is always called

Raising <class 'IndexError'>
Caught some other error: IndexError
this clean up code is always called

Raising None
no exception raised
this code called if there is no exception
this clean up code is always called


## Exception hierarchy

- when we use `except: ` clause without specifying any type of exception, it'll catch all subclasses of BaseException
- always catch specific Exception type explictly
    - except: without type is an error and will flag it in code review
![Exception Hieararchy](resources/ExceptionHierarchy.png)

## Defining your own Exception

- if the Python provided built-in exception classes are not adquate, you can create your own
- create a sub class that inherits from Exception class

In [40]:
class InvalidWithdrawl(ValueError):
    pass

In [41]:
raise InvalidWithdrawl("You don't have $50 in your account")

InvalidWithdrawl: You don't have $50 in your account

In [42]:
# more complete example
from decimal import Decimal

class InvalidWithdrawl(ValueError):
    def __init__(self, balance:Decimal, amount:Decimal) ->None:
        super().__init__(f"account doesn't have {amount} amount")
        self.amount = amount
        self.balance = balance
        
    # how overdrawn the withdraw request is  
    def overage(self) -> Decimal:
        return self.amount - self.balance
    
    

In [43]:
raise InvalidWithdrawl(Decimal('25.00'), Decimal('50.00'))

InvalidWithdrawl: account doesn't have 50.00 amount

In [49]:
try:
    balance = Decimal('25.00')
    withdraw = Decimal('50.00')
    raise InvalidWithdrawl(balance, withdraw)
except InvalidWithdrawl as ex:
    print("I'm sorry, but your withdrawl "
          " is more than your balance by "
         f'{ex.overage()}')

I'm sorry, but your withdrawl  is more than your balance by 25.00


## When to raise Exceptions

- should you use `if` statements for checking unknowns or just execute code using try... except and see what happens?
- Python developers tend to follow model: **EAFP**
    - **It's easier to ask for forgiveness than permission**
    - execute code then deal with anything that goes wrong
- less popular model: **LBYL**
    - **Look Before You Leap**
    - you're burning some CPU cyles looking for unusual situation that is not going to arise in the normal path through the code
- it is wise to use exceptions for exceptional circumstances, even those circumstances are only a little bit exceptional
- e.g, the following two functions are identical!

In [51]:
def divide_with_exception(dividend: int, divisor: int) -> None:
    try:
        print(f'{dividend/divisor=}')
    except ZeroDivisionError:
        print("You can't divide by zero")

def divide_with_if(dividend: int, divisor: int) -> None:
    if divisor == 0:
        print("You can't divide by zero")
    else:
        print(f"{dividend/divisor=}")

In [52]:
divide_with_exception(10, 0)

You can't divide by zero


In [53]:
divide_with_if(10, 0)

You can't divide by zero


## Exercises

- Solve the following Kattis Problems using OOD
- Must write atleast one Exception class and use it to solve the problem using OOP

1. A New Alphabet - https://open.kattis.com/problems/anewalphabet
2. Babelfish - https://open.kattis.com/problems/babelfish